In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.7/718.7 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-7.0.0-py3-none-any.whl size=21129 sha256=7afeaf27dd65e5beec42fa1687d2e016e94e25b437aff496f53c6b2e8f667c5a
  Stored in directory: /root/.cache/pip/wheels/60/29/7b/f64332aa7e5e88fbd56d4002185ae22dcdc83b35b3d1c2cbf5
Successfully built pyngrok


In [3]:
!pip install flask pandas tensorflow transformers flask-ngrok matplotlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.1 MB/s eta 0:00:00


In [4]:
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok

deb https://ngrok-agent.s3.amazonaws.com buster main
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://ngrok-agent.s3.amazonaws.com buster InRelease [20.3 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [498 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [998 kB]
Get:12 https://ngrok-agent.s3.amazonaws.com buster/main amd64 Pac

In [5]:
!ls

drive  sample_data


In [6]:
!ngrok authtoken "2T6WyKOJsq5hUIcFqTiepcwaCKx_88gEw17DGocLN7PKiA4Z6"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [7]:
pip install plotly


In [8]:
pip install wordcloud


In [ ]:
from flask import Flask, request, render_template, send_file, make_response
import pandas as pd
import tensorflow as tf
import os
import tempfile
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
from pyngrok import ngrok
import json
import plotly.express as px

app = Flask(__name__, template_folder='/content/drive/MyDrive/Project/templates')

# Load the model and tokenizer
model_path = '/content/drive/MyDrive/Project/Model'  # Replace with the path to your saved model
tokenizer_path = '/content/drive/MyDrive/Project/Tokenizer'  # Replace with the path to your saved tokenizer

model = TFRobertaForSequenceClassification.from_pretrained(model_path)
tokenizer = RobertaTokenizer.from_pretrained(tokenizer_path)

# Load the label-to-id mapping
label2id = {'positive': 2, 'neutral': 1, 'negative': 0}


def preprocess_data(file_path, review_column='Review'):
    if file_path.lower().endswith('.csv'):
        df = pd.read_csv(file_path, encoding='unicode_escape')
    elif file_path.lower().endswith(('.xls', '.xlsx')):
        df = pd.read_excel(file_path, encoding='unicode_escape')
    else:
        raise ValueError("Unsupported file format. Only CSV and Excel files are supported.")

    df = df.dropna(subset=[review_column])  # Remove missing review values
    encodings = tokenizer(list(df[review_column]), truncation=True, padding=True, return_tensors='tf')
    return df, encodings

@app.route('/')
def index():
    return render_template('upload.html')

@app.route('/analyze_sentiment', methods=['POST'])
def analyze_sentiment():
    file = request.files['file']
    if file:
        file_path = os.path.join(tempfile.gettempdir(), "uploaded_file.csv")
        file.save(file_path)

        try:
            df, encodings = preprocess_data(file_path)
        except ValueError as e:
            return str(e)

        # Predict the sentiment of the input reviews
        input_ids = encodings['input_ids']
        attention_mask = encodings['attention_mask']
        predictions = model.predict({'input_ids': input_ids, 'attention_mask': attention_mask})
        predicted_labels = tf.argmax(predictions.logits, axis=1).numpy()

        # Convert predicted labels to sentiment
        predicted_sentiments = [list(label2id.keys())[list(label2id.values()).index(label)] for label in predicted_labels]

        # Add the sentiment column to the DataFrame
        df['Sentiment'] = predicted_sentiments

        # Calculate the percentage of positive, negative, and neutral sentiments for each product
        product_sentiment_percentage = df.groupby(['Product'])['Sentiment'].value_counts(normalize=True).unstack().fillna(0) * 100

        # Save the DataFrame with sentiment as separate CSV files for each product
        for product, group_df in df.groupby('Product'):
            product_result_file_path = os.path.join(tempfile.gettempdir(), f"{product}_with_sentiment.csv")
            group_df.to_csv(product_result_file_path, index=False)

        # Save the DataFrame with sentiment as the overall sentiment file with the original file name
        overall_result_file_path = os.path.join(tempfile.gettempdir(), f"result_with_sentiment_{file.filename}")
        df.to_csv(overall_result_file_path, index=False)

        # Create donut graphs for each product
        donut_graphs = []
        for product, sentiments in product_sentiment_percentage.iterrows():
            fig = px.pie(sentiments, values=sentiments.values, names=sentiments.index, hole=0.4)
            donut_graphs.append(fig.to_json())

        # Create bar chart for sentiment comparison between products
        fig_bar_chart = px.bar(product_sentiment_percentage.reset_index(), x='Product', y=['positive', 'negative', 'neutral'], barmode='group', title="Sentiment Comparison between Products")

        # Convert the bar chart to JSON format to pass to the template
        bar_chart_json = fig_bar_chart.to_json()

        return render_template(
            'result.html',
            product_sentiments=product_sentiment_percentage.reset_index().to_dict(orient='records'),
            overall_sentiment_percentage={
                'positive': round(product_sentiment_percentage['positive'].mean(), 2),
                'negative': round(product_sentiment_percentage['negative'].mean(), 2),
                'neutral': round(product_sentiment_percentage['neutral'].mean(), 2)
            },
            download_link=f"/download_result/{file.filename}",
            donut_graphs=donut_graphs,
            bar_chart_json=bar_chart_json
        )

    return "File upload failed!"

@app.route('/download_result/<filename>')
def download_result(filename):
    result_file_path = os.path.join(tempfile.gettempdir(), f"result_with_sentiment_{filename}")
    if os.path.exists(result_file_path):
        response = make_response(send_file(result_file_path, as_attachment=True, download_name=f"result_with_sentiment_{filename}"))
        response.headers["Content-Disposition"] = f"attachment; filename=result_with_sentiment_{filename}"
        return response
    return "File not found!"

@app.route('/download_product_result/<product>')
def download_product_result(product):
    result_file_path = os.path.join(tempfile.gettempdir(), f"{product}_with_sentiment.csv")
    if os.path.exists(result_file_path):
        return send_file(result_file_path, as_attachment=True, download_name=f"{product}_with_sentiment.csv")
    return "File not found!"

def run_flask_with_ngrok(app):
    # Get the Flask app's port
    port = 5000

    # Start ngrok and create a tunnel to the Flask app on the specified port
    public_url = ngrok.connect(port)
    print('Public URL:', public_url)

    try:
        app.run(port=port)

    except KeyboardInterrupt:
        print('Shutting down...')
        ngrok.kill()

if __name__ == '__main__':
    run_flask_with_ngrok(app)


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/Project/Model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


Public URL: NgrokTunnel: "https://151e-35-247-61-40.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [21/Sep/2023 14:37:39] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Sep/2023 14:37:40] "GET /favicon.ico HTTP/1.1" 404 -
